<a href="https://colab.research.google.com/github/sadra-barikbin/persian-information-retrieval-example/blob/main/Persian-IR-example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
!pip install hazm transformers ir_measures

     |████████████████████████████████| 316 kB 5.3 MB/s 
     |████████████████████████████████| 3.4 MB 47.9 MB/s 
     |████████████████████████████████| 41 kB 292 kB/s 
     |████████████████████████████████| 1.4 MB 39.1 MB/s 
     |████████████████████████████████| 233 kB 35.6 MB/s 
     |████████████████████████████████| 596 kB 33.0 MB/s 
     |████████████████████████████████| 895 kB 35.2 MB/s 
     |████████████████████████████████| 61 kB 451 kB/s 
     |████████████████████████████████| 3.3 MB 27.8 MB/s 
     |████████████████████████████████| 291 kB 36.8 MB/s 
     |████████████████████████████████| 59 kB 5.4 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394484 sha256=c1eadaab582b081dde18974212c2dc85bf9162fca6ea129d67fa48169ac9e521
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154646 sha256=cc7092408b965

In [2]:
import torch
import yaml
import numpy as np
import pandas as pd
import ir_measures as IRm
from typing import List
from pathlib import Path
from sklearn.metrics import make_scorer, average_precision_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from transformers import AutoTokenizer, AutoModelForMaskedLM

# Loading & Preparing Data

## Corpus

In [3]:
!wget https://github.com/language-ml/2-LM-embedding-projects/raw/main/problem3/doc_collection.zip

--2021-12-30 17:01:36--  https://github.com/language-ml/2-LM-embedding-projects/raw/main/problem3/doc_collection.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/language-ml/2-LM-embedding-projects/main/problem3/doc_collection.zip [following]
--2021-12-30 17:01:36--  https://raw.githubusercontent.com/language-ml/2-LM-embedding-projects/main/problem3/doc_collection.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6083582 (5.8M) [application/zip]
Saving to: ‘doc_collection.zip’

doc_collection.zip  100%[===================>]   5.80M  --.-KB/s    in 0.1s    

2021-12-30 17:01:36 (58.2 MB/s) -

In [4]:
!unzip doc_collection.zip

Archive:  doc_collection.zip
   creating: IR_dataset/
  inflating: IR_dataset/559.txt      
  inflating: IR_dataset/3112.txt     
  inflating: IR_dataset/2206.txt     
  inflating: IR_dataset/243.txt      
  inflating: IR_dataset/1007.txt     
  inflating: IR_dataset/839.txt      
  inflating: IR_dataset/1330.txt     
  inflating: IR_dataset/757.txt      
  inflating: IR_dataset/525.txt      
  inflating: IR_dataset/1113.txt     
  inflating: IR_dataset/2734.txt     
  inflating: IR_dataset/1999.txt     
  inflating: IR_dataset/2916.txt     
  inflating: IR_dataset/2209.txt     
  inflating: IR_dataset/1030.txt     
  inflating: IR_dataset/1317.txt     
  inflating: IR_dataset/1325.txt     
  inflating: IR_dataset/437.txt      
  inflating: IR_dataset/1046.txt     
  inflating: IR_dataset/2482.txt     
  inflating: IR_dataset/1761.txt     
  inflating: IR_dataset/2911.txt     
  inflating: IR_dataset/162.txt      
  inflating: IR_dataset/2298.txt     
  inflating: IR_dataset/2982.txt  

In [5]:
!cat IR_dataset/1000.txt

ببر سیبری که با نام‌های ببر آلتایی، ببر منچوری، ببر کره‌ای، ببر آمور و ببر اوسوری نیز شناخته می‌شود، یکی از زیرگونه‌های ببر است که در گذشته در بخش‌های وسیعی از شرق آسیا می‌زیست اما امروزه تنها در منطقهٔ حفاظت شده‌ای در شرق سیبری زندگی می‌کند. ببر سیبری بزرگترین زیرگونهٔ ببر و بزرگترین گربه‌سان زندهٔ جهان است. ببر منقرض شده مازندران نزدیک‌ترین زیرگونه ببر به ببر سیبری است و مطالعات ژنتیکی جدید حکایت از آن دارد که این دو را حتی می‌توان یک زیرگونه محسوب کرد.

ببر سیبری در دهه ۱۹۳۰ در آستانه انقراض قرار داشت و تعداد آن‌ها تنها به بیست تا سی ببر کاهش یافته بود. اما این حیوان به طرزی باورنکردنی از انقراض قریب‌الوقوع رهایی جست و جمعیت آن تا سال ۲۰۱۰ به حدود ۳۶۰ ببر رسید. ببر سیبری با توجه به همین افزایش جمعیت از سال ۲۰۱۰ از بالاترین ردهٔ حفاظتی یعنی «به شدت در معرض خطر» خارج شده و در یک رده پایین‌تر یعنی «در خطر انقراض» قرار گرفته است. ببرهای سیبری تنوع ژنتیکی بسیار پائینی دارند که این به دلیل کاهش شدید جمعیت این حیوان در دهه ۱۹۴۰ و تعداد اندک توله ببرهایی است که به بلوغ می‌رسند. ضمن اینکه بی

In [6]:
corpus = [(path.stem, path.open().read()) for path in Path('IR_dataset').iterdir()]
corpus = pd.DataFrame(corpus, columns=['docId','text'])

In [7]:
corpus

,docId,text
0,1181,درگذار از سده‌ها زبان فنلاندی وام‌واژه‌هایی بس...
1,1206,بر مبنای یافته‌های باستان‌شناسان شهر سوخته ۲۸۰...
2,1687,زمانی که کمیته بین‌المللی المپیک (آی.اُ. سی) د...
3,623,در جهان اسلام کتیبه‌نگاری در بناهای یادبود، مق...
4,1688,مسجد جامع دهلی دارای دو محراب باشکوه و حیاط وس...
...,...,...
3253,2427,علمای گذشته که از کاشفی یاد کرده‌اند، چه صاحب‌...
3254,1247,لاووازیه اولین آزمایش‌های کمی مناسب در مورد اک...
3255,1381,"* ""پژوهش‌های ایران‌شناسی نامواره دکتر محمود اف..."
3256,1088,انجمن حجتیه پس از کودتای ۱۳۳۲ با هدف تعلیم کاد...


## Qrels

In [8]:
!wget https://raw.githubusercontent.com/language-ml/2-LM-embedding-projects/main/problem3/evaluation_IR.yml

--2021-12-30 17:01:38--  https://raw.githubusercontent.com/language-ml/2-LM-embedding-projects/main/problem3/evaluation_IR.yml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 50854 (50K) [text/plain]
Saving to: ‘evaluation_IR.yml’

evaluation_IR.yml   100%[===================>]  49.66K  --.-KB/s    in 0.01s   

2021-12-30 17:01:38 (5.06 MB/s) - ‘evaluation_IR.yml’ saved [50854/50854]



In [9]:
query_data = yaml.safe_load(open('evaluation_IR.yml'))

In [10]:
query = list(query_data.keys())
relevant = [query_data[k]['relevant'][0] for k in query_data]
qrels = [{'query_id':q, 'doc_id':d,
          'relevance':3} for q in query for d in query_data[q]['similar_high']]
qrels.extend([{'query_id':q, 'doc_id':d,
          'relevance':2} for q in query for d in query_data[q]['similar_med']])
qrels = [{'query_id':q, 'doc_id':d,
          'relevance':1} for q in query for d in query_data[q]['similar_low']]
qrels.extend([{'query_id':q, 'doc_id':query_data[q]['relevant'], 'relevance':4} for q in query])
qrels = pd.DataFrame(qrels)

In [11]:
qrels.head()

,query_id,doc_id,relevance
0,آدولف هیتلر شکست و مرگ,355,1
1,آدولف هیتلر شکست و مرگ,356,1
2,آدولف هیتلر شکست و مرگ,357,1
3,آدولف هیتلر شکست و مرگ,358,1
4,آدولف هیتلر شکست و مرگ,359,1


In [12]:
res = {
    'q0':{
        'doc0': 1.,
        'doc1': 2.,
        'doc3': 5.
    },
    'q1':{
        'doc0': 4.,
        'doc1': 6.,
        'doc2': 2.
    }
}
qrels = {
    'q0':{
        'doc0': 1,
        'doc1': 1,
        'doc3': 1
    },
    'q1':{
        'doc0': 0,
        'doc1': 2,
        'doc2': 2
    }
}
IRm.calc_aggregate([IRm.AP(rel=0)],qrels,res)

TypeError: ignored

## Normaliztion

# Embedding the documents

## Method 1 : Tfidf

In [13]:
vectorizer = TfidfVectorizer(max_features=500,ngram_range=(1,2))
vectorizer.fit(corpus.text)

TfidfVectorizer(max_features=500, ngram_range=(1, 2))

## Method 2 : Neural Network (RoBERTa)

In [ ]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
class RoBERTaVectorizer(TransformerMixin):
  def __init__(self):
    super().__init__()
    self.tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/roberta-fa-zwnj-base")
    self.model = AutoModelForMaskedLM.from_pretrained("HooshvareLab/roberta-fa-zwnj-base")\
                                     .to(DEVICE)
  def fit(self,x):
    pass
  def transform(self,x: List[str]):
    encoded_batch = self.tokenizer.batch_encode_plus(x, return_tensors='pt', padding=True)
    encoded_batch = {k: v.to(DEVICE) for k, v in encoded_batch.items()}
    with torch.no_grad():
      output = self.model(**encoded_batch)
    return output

## Method 3: ParsBert

In [12]:
!pip install -q transformers
!pip install -q hazm
!pip install -q clean-text[gpl]

     |████████████████████████████████| 64 kB 1.7 MB/s 
     |████████████████████████████████| 170 kB 12.1 MB/s 
     |████████████████████████████████| 235 kB 29.0 MB/s 


In [17]:
from transformers import AutoConfig, AutoTokenizer, AutoModel, TFAutoModel

model_name_or_path = "HooshvareLab/bert-fa-zwnj-base"
config = AutoConfig.from_pretrained(model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

model = AutoModel.from_pretrained(model_name_or_path)
model = model.cuda()

Some weights of the model checkpoint at HooshvareLab/bert-fa-zwnj-base were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at HooshvareLab/bert-fa-zwnj-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.poo

In [18]:
text = "گور بابای درس" 
encoding = tokenizer.encode_plus(
      text,
      add_special_tokens=True, # Add '[CLS]' and '[SEP]'
      return_token_type_ids=False,
      max_length = 500,
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt',  # Return PyTorch tensors
    )
out = model(
            input_ids = encoding['input_ids'].cuda(), 
            attention_mask= encoding['attention_mask'].cuda())
out['pooler_output'][0].shape

torch.Size([768])

In [27]:
def get_embed(part):
  encoding = tokenizer.encode_plus(
    part,
    add_special_tokens=True, # Add '[CLS]' and '[SEP]'
    return_token_type_ids=False,
    max_length = 500,
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt',  # Return PyTorch tensors
  )
  out = model(
      input_ids = encoding['input_ids'].cuda(), 
      attention_mask= encoding['attention_mask'].cuda())
  return out['pooler_output'].cpu().detach().numpy()

In [29]:
doc_vec = np.zeros((1, 768))
doc_map = np.zeros(1)
import tqdm

for index, doc in tqdm.tqdm(corpus.iterrows()):
  doc_split = doc['text'].split()
  doc_parts = [' '.join(doc_split[i:i + 300]) for i in range(0, len(doc_split) - 150, 150)]
  for part in doc_parts:
    doc_vec = np.append(doc_vec, get_embed(part), axis = 0)
    doc_map = np.append(doc_map, doc['docId'])


3258it [13:06,  4.14it/s]


# Document Retrieval

In [44]:
class KNN_based_IR(BaseEstimator):
  def __init__(self,n_neighbors=1+10+10+10) -> None:
    super().__init__()
    self.nn = NearestNeighbors(n_neighbors=n_neighbors)
  def set_params(self,**kwargs):
    self.nn.set_params(**kwargs)
  def fit(self, X, y):
    self.nn.fit(X)
  def predict(self, X):
    scores, docIds = self.nn.kneighbors(X)
    return scores, docIds

In [45]:
bert_knn = KNN_based_IR(50)
bert_knn.fit(doc_vec, None)

# IR Evaluation
Tailored for our multi-level Test Collection.

In [47]:
bert_pred = []
for q, ret in tqdm.tqdm(query_data.items()):
  pr = get_embed(q)
  res = [doc_map[i] for i in bert_knn.predict(pr)[1][0] if i != 0]
  bert_pred.append(res)

100%|██████████| 150/150 [00:07<00:00, 20.26it/s]


## Adapting IR output to our Test Collection

In [ ]:
class IRoutputToHW4TestDataAdapter(TransformerMixin):
  pass

## MRR (Mean Reciprocal Rank)

In [ ]:
MRR = IRm.measures.MRR()
def mrr(qrels, ret):
  return np.mean([MRR.calc_aggregate(qrels[qrels.relevance == level],
                                     ret[ret.relevance == level]) for level in range(1,4+1)])
mrr_scorer = make_scorer(mrr)

## MAP (Mean Average Precision)

In [ ]:
def map(qrels, ret):
  return np.mean([average_precision_score(qrels[qrels.relevance == level],
                        ret[ret.relevance == level],average='samples') for level in range(1,4+1)])

map_scorer = make_scorer(map)

## P@K

# Pipeline Definition

In [ ]:
pipeline = Pipeline([('embedding','passthrough'),
                     ('retrieval','passthrough')])